In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
productURL = input("Enter the product URL\n")

Enter the product URL
https://www.flipkart.com/samsung-crystal-4k-neo-series-108-cm-43-inch-ultra-hd-4k-led-smart-tizen-tv-voice-search/p/itma076c4dd16aa4?pid=TVSGEAGP9GXWCHCX&fm=neo%2Fmerchandising&ppt=dynamic&ppn=dynamic&ssid=gg6rurvu800000001689592474429


In [3]:
main_product = requests.get(productURL)
print(main_product.status_code)
if (main_product.status_code)==200:
    soup = BeautifulSoup(main_product.text, 'html.parser')

200


In [4]:
# Getting the partial link for the 1st review page
main_div = soup.find('div', class_='JOpGWq')
links = main_div.find_all('a', recursive=False)
link_urls = [link['href'] for link in links]
partial_link=link_urls[-1]

In [5]:
review_link = 'https://www.flipkart.com'+partial_link
review_link

'https://www.flipkart.com/samsung-crystal-4k-neo-series-108-cm-43-inch-ultra-hd-4k-led-smart-tizen-tv-voice-search/product-reviews/itma076c4dd16aa4?pid=TVSGEAGP9GXWCHCX&lid=LSTTVSGEAGP9GXWCHCXLNHH4M&marketplace=FLIPKART'

In [6]:
review_page = requests.get(review_link)
# print(review_page.status_code)
if review_page.status_code==200:
    soup = BeautifulSoup(review_page.content, 'html.parser')
    div = soup.find('div', class_='_2MImiq _1Qnn1K')
    span = div.find('span')
    if span:
        span_text = span.text
        print(span_text)

Page 1 of 154


In [7]:
number_of_pages = int(span_text.split()[-1])
number_of_pages

154

In [8]:
nav = div.find('nav', class_='yFHi8N')
link = nav.find('a')
if link:
    href = link.get('href')
    print(href) # Partial link of the 1st review page and general one

/samsung-crystal-4k-neo-series-108-cm-43-inch-ultra-hd-4k-led-smart-tizen-tv-voice-search/product-reviews/itma076c4dd16aa4?pid=TVSGEAGP9GXWCHCX&lid=LSTTVSGEAGP9GXWCHCXLNHH4M&marketplace=FLIPKART&page=1


In [9]:
review_page_1_gen_link = 'https://www.flipkart.com'+href
review_page_1_gen_link

'https://www.flipkart.com/samsung-crystal-4k-neo-series-108-cm-43-inch-ultra-hd-4k-led-smart-tizen-tv-voice-search/product-reviews/itma076c4dd16aa4?pid=TVSGEAGP9GXWCHCX&lid=LSTTVSGEAGP9GXWCHCXLNHH4M&marketplace=FLIPKART&page=1'

### We need to get number of stars,headline,entire review,date

In [10]:
url = review_page_1_gen_link

In [11]:
# !pip install selenium

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [13]:
# driver = webdriver.Chrome() 
# driver.get(url)
# # div = driver.find_elements(By.CSS_SELECTOR, 'div._27M-vq._2hwual')
# # for i in div:
# #     i.click()
# elements = driver.find_elements(By.CSS_SELECTOR, 'div._27M-vq._2hwual')  
# for element in elements:
#     try:
#         WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div._27M-vq._2hwual')))
#         element.click()
#     except Exception as e:
#         print(f"Failed to click on element: {e}")

In [14]:
# span = driver.find_elements(By.CSS_SELECTOR, 'span._1BWGvX')
# for i in span:
#     actions = ActionChains(driver)
#     actions.move_to_element(i)
#     actions.perform()
#     i.click()

In [15]:
# #Get rating
# div = driver.find_elements(By.CSS_SELECTOR, 'div._3LWZlK._1BLPMq')
# ratings = []
# for i in div:
#     ratings.append(i.text)
# print(ratings)

In [16]:
# #For title
# paragraph = driver.find_elements(By.CSS_SELECTOR, 'p._2-N8zT')
# titles = []
# for i in paragraph:
#     titles.append(i.text)
# print(titles)

In [17]:
# len(titles)

In [18]:
# #For main review
# div = driver.find_elements(By.CSS_SELECTOR, 'div.t-ZTKy')
# main_reviews=[]
# for i in div:
#     review = i.text
#     main_reviews.append(review.replace('\n', ' ').strip())
# print(main_reviews)

In [19]:
# driver.quit()

In [20]:
file_name = soup.title.text[:15].lower().replace(' ','_')
file_name

'samsung_crystal'

In [21]:
import pandas as pd

In [22]:
from tqdm import tqdm

In [26]:
#Getting the data frame for 10 pages of reviews
reviews_df = pd.DataFrame()
driver = webdriver.Chrome() 
for i in tqdm(range(1,min(50,number_of_pages)+1)):
    url = review_page_1_gen_link[:-1]+str(i)

    driver.get(url)
    elements = driver.find_elements(By.CSS_SELECTOR, 'div._27M-vq._2hwual')  
    for element in elements:
        try:
            WebDriverWait(driver, 15,poll_frequency=1).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div._27M-vq._2hwual')))
            element.click()
        except Exception as e:
            print(f"Failed to click on element: {e}")
    #Clicking on all the read more
    span = driver.find_elements(By.CSS_SELECTOR, 'span._1BWGvX')
    for i in span:
        actions = ActionChains(driver)
        actions.move_to_element(i)
        actions.perform()
        i.click()
    #Get rating
    div = driver.find_elements(By.CSS_SELECTOR, 'div._3LWZlK._1BLPMq')
    ratings = []
    for i in div:
        ratings.append(i.text)
    #The data is unavailable only dummy page is present.
    if(len(ratings)==0):
        break
    #For title
    paragraph = driver.find_elements(By.CSS_SELECTOR, 'p._2-N8zT')
    titles = []
    for i in paragraph:
        titles.append(i.text)
    #For main review
    div = driver.find_elements(By.CSS_SELECTOR, 'div.t-ZTKy')
    main_reviews=[]
    for i in div:
        review = i.text
        main_reviews.append(review.replace('\n', ' ').strip())
    reviews_dict = {'ratings':ratings,'titles':titles,'reviews':main_reviews}
    temp_df = pd.DataFrame(reviews_dict)
    reviews_df = reviews_df.append(temp_df,ignore_index=True)
driver.quit()

 30%|████████████████████████▌                                                         | 15/50 [00:48<01:53,  3.24s/it]


In [27]:
reviews_df.to_csv(f'data_files\{file_name}_reviews.csv',index=False)